# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f83242d97c0>
├── 'a' --> tensor([[-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533]])
└── 'x' --> <FastTreeValue 0x7f83242d9c10>
    └── 'c' --> tensor([[ 1.4952,  0.2560, -1.1297, -0.9023],
                        [ 2.2990, -0.4911, -0.6080,  0.4507],
                        [ 0.4259, -0.9293,  1.5505,  0.9967]])

In [4]:
t.a

tensor([[-0.5430, -0.6403, -0.5916],
        [-2.4478,  0.4527,  0.6533]])

In [5]:
%timeit t.a

65.1 ns ± 0.0565 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f83242d97c0>
├── 'a' --> tensor([[ 1.1717, -1.0576,  0.5091],
│                   [-1.1598,  2.0621, -1.5083]])
└── 'x' --> <FastTreeValue 0x7f83242d9c10>
    └── 'c' --> tensor([[ 1.4952,  0.2560, -1.1297, -0.9023],
                        [ 2.2990, -0.4911, -0.6080,  0.4507],
                        [ 0.4259, -0.9293,  1.5505,  0.9967]])

In [7]:
%timeit t.a = new_value

68.2 ns ± 0.0715 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.5430, -0.6403, -0.5916],
               [-2.4478,  0.4527,  0.6533]]),
    x: Batch(
           c: tensor([[ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967]]),
       ),
)

In [10]:
b.a

tensor([[-0.5430, -0.6403, -0.5916],
        [-2.4478,  0.4527,  0.6533]])

In [11]:
%timeit b.a

58.2 ns ± 0.114 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5860,  0.2403,  0.4302],
               [-3.4232, -1.6079,  0.3069]]),
    x: Batch(
           c: tensor([[ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.206 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

818 ns ± 0.259 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 40.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

148 µs ± 512 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8277f947c0>
├── 'a' --> tensor([[[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]],
│           
│                   [[-0.5430, -0.6403, -0.5916],
│                    [-2.4478,  0.4527,  0.6533]]])
└── 'x' --> <FastTreeValue 0x7f8277f94340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.4 µs ± 91.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f82785cb940>
├── 'a' --> tensor([[-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533],
│                   [-0.5430, -0.6403, -0.5916],
│                   [-2.4478,  0.4527,  0.6533]])
└── 'x' --> <FastTreeValue 0x7f8277fb2f40>
    └── 'c' --> tensor([[ 1.4952,  0.2560, -1.1297, -0.9023],
                        [ 2.2990, -0.4911, -0.6080,  0.4507],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 78.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 65.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.4952,  0.2560, -1.1297, -0.9023],
                       [ 2.2990, -0.4911, -0.6080,  0.4507],
                       [ 0.4259, -0.9293,  1.5505,  0.9967]],
              
                      [[ 1.4952,  0.2560, -1.1297, -0.9023],
                       [ 2.2990, -0.4911, -0.6080,  0.4507],
                       [ 0.4259, -0.9293,  1.5505,  0.9967]],
              
                      [[ 1.4952,  0.2560, -1.1297, -0.9023],
                       [ 2.2990, -0.4911, -0.6080,  0.4507],
                       [ 0.4259, -0.9293,  1.5505,  0.9967]],
              
                      [[ 1.4952,  0.2560, -1.1297, -0.9023],
                       [ 2.2990, -0.4911, -0.6080,  0.4507],
                       [ 0.4259, -0.9293,  1.5505,  0.9967]],
              
                      [[ 1.4952,  0.2560, -1.1297, -0.9023],
                       [ 2.2990, -0.4911, -0.6080,  0.4507],
                       [ 0.4259, -0.9293,  1.5505,  0.9967]],

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 94.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967],
                      [ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967],
                      [ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967],
                      [ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967],
                      [ 1.4952,  0.2560, -1.1297, -0.9023],
                      [ 2.2990, -0.4911, -0.6080,  0.4507],
                      [ 0.4259, -0.9293,  1.5505,  0.9967],
                      [ 1.4952,  0.2560, -1.1297, -0.9023],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 459 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

319 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
